### Supisara Poopair ID: 6410422024

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
#import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA


from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict,GridSearchCV,RandomizedSearchCV
from sklearn import preprocessing,metrics
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,roc_curve, auc, f1_score, precision_score, recall_score, roc_auc_score,mean_squared_error
from sklearn.preprocessing import StandardScaler



In [4]:
df_train_og = pd.read_csv('/kaggle/input/dads6003-in-class-competition/train.csv')
df_train_og.head()

In [5]:
df_test = pd.read_csv('/kaggle/input/dads6003-in-class-competition/test.csv')
#df_test.head()

In [6]:
# Your work
# Must have
# 1. Cleansing data
# 2. Exploratory Data Analysis (describe insight and visualization)
# 3. Training data - Crossvalidation
# 4. Testing data

# 1. Cleansing Data

In [7]:
df_train_og.isna().sum().sort_values(ascending = False)

In [8]:
df_train_og.describe()

In [9]:
df_train = df_train_og.copy() 
#df_train = df_train.fillna(df_train.mean())
df_train = df_train.fillna(df_train.median())
#df_train = df_train.fillna(0)
#df_train = df_train.dropna()
df_train.shape

In [10]:
df_train.describe()

# 2. Exploring Data

In [11]:
C1 = len(df_train[df_train.y == 1])
C0 = len(df_train[df_train.y == 0])
print('There are '+ str(C1) + ' 1-class in the original dataset, even though there are ' + str(C0) +' of 0-class in the dataset.')

sns.countplot(data =df_train , x ='y')

In [12]:
correlation = df_train.corr()

f, ax = plt.subplots(figsize=(20,15))
plt.title('Correlation of numerical attributes', size=16)
sns.heatmap(correlation, lw=0.5, cmap='coolwarm', annot=True)
plt.show()


In [13]:
fig = plt.figure(figsize=(15,20))
i = 0
for column in df_train:
    sub = fig.add_subplot(6,4,i+1)
    sub.set_xlabel(column)
    df_train[column].plot(kind = 'hist')
    i = i+1

In [14]:
fig = plt.figure(figsize=(15,20))
i = 0
for column in df_train:
    sub = fig.add_subplot(6,4,i+1)
    sns.boxplot(x = 'y', y = column, data = df_train)
    i = i+1

# 3. Training data - Crossvalidation

In [15]:
X = df_train.drop(['y'], axis=1)
y = df_train['y']

In [16]:
#Scaling

sc = StandardScaler()
X_sc = sc.fit_transform(X)

#X_sc

In [17]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()
model.fit(X_sc,y)

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='bar',color='darkblue')
plt.show()

In [18]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

fs = SelectKBest(score_func=f_classif, k=15)

X_selected = fs.fit_transform(X_sc, y)


print(X_selected.shape)

In [19]:
print('Selected Features are')

print(fs.get_feature_names_out())

In [20]:
df_sel = pd.DataFrame(X_selected)
fig = plt.figure(figsize=(15,20))
i = 0
for column in df_sel:
    sub = fig.add_subplot(6,4,i+1)
    sub.set_xlabel(column)
    df_sel[column].plot(kind = 'hist')
    i = i+1

In [21]:
per_test = 0.25

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=per_test, random_state=101)

---

**Train**

In [22]:
model_pipeline = []

model_pipeline.append(LogisticRegression())
model_pipeline.append(KNeighborsClassifier())
model_pipeline.append(DecisionTreeClassifier())
model_pipeline.append(RandomForestClassifier())
model_pipeline.append(SVC())
model_pipeline.append(MLPClassifier())
model_pipeline.append(GaussianNB())

In [23]:
model_list = ['Log Reg','KNN','Desision Tree','Random Forrest','SVC','NN','GNB']
acc_list = []
auc_list = []
cm_list = []
f1_list = []

for model in model_pipeline:
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    acc_list.append(metrics.accuracy_score(y_test,y_pred))
    fpr,tpr,_tresholds = metrics.roc_curve(y_test,y_pred)
    auc_list.append(round(metrics.auc(fpr,tpr),2))
    cm_list.append(confusion_matrix(y_test,y_pred))
    f1_list.append(f1_score(y_test, y_pred, average='weighted'))

In [24]:
result_df = pd.DataFrame({'Model':model_list,'Accuracy':acc_list,'AUC':auc_list,'F1':f1_list})
result_df

In [25]:
fig = plt.figure(figsize = (18,10))
for i in range(len(cm_list)):
    cm = cm_list[i]
    model = model_list[i]
    sub = fig.add_subplot(3,3,i+1).set_title(model)
    cm_plot = sns.heatmap(cm,annot = True,cmap='Blues_r')
    cm_plot.set_xlabel('Predicted Values')
    cm_plot.set_ylabel('Actual Values')

---

Best model with highest F1-Score is Neural Network

<break>**Tuning Model using Randomized**

In [26]:
param_grid = {
    'hidden_layer_sizes': [(150,100,50), (120,80,40), (100,50,30),(50,25,10)],
    'max_iter': [50, 100, 150,300,500,1500],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive'],
}

grid = RandomizedSearchCV(MLPClassifier(), param_grid,cv=5)

In [27]:
grid.fit(X_train,y_train)

In [35]:
best_grid = grid.best_estimator_

In [29]:
print(grid.best_params_)

print(grid.best_estimator_)

In [45]:
best_grid = MLPClassifier(activation='tanh', max_iter=150, hidden_layer_sizes=(150, 100, 50)).fit(X_train,y_train)

In [46]:
y_pred_2 = None
y_pred_2 = best_grid.predict(X_test)


print('F1 - Accuracy: {:.5f}'.format(f1_score(y_test, y_pred_2, average='weighted')))

In [47]:
print(classification_report(y_test, y_pred_2))

# 4. Testing

In [ ]:
df_test_sc = sc.transform(df_test)
df_test_kb = fs.transform(df_test_sc)

predict = best_grid.predict(df_test_kb)

fields = ['id','Expected']
df = pd.DataFrame()
id = list(range(1,2501))
df['id']=pd.Series(id)
df['Expected']=pd.Series(predict)
df.to_csv('submit_final.csv', index=False)
print('done')